In [104]:
import ReadData as rd
# import ReviewAnalysis as ra
import gzip
from collections import defaultdict
import pandas as pd
from wordcloud import STOPWORDS, WordCloud
import matplotlib.pyplot as plt
from collections import defaultdict

from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
import gensim
import numpy as np
import nltk
import math
from nltk.stem import WordNetLemmatizer
lemm = WordNetLemmatizer()


reviewFile="dataset/reviewShuffled.json"
businessFile="dataset/business.json"
userFile="dataset/user.json"

In [47]:
def getPrediction(alpha,uB,iB,user,item,y_u,y_i,uMap,iMap):
    i=uMap[user] if user in uMap else -1
    j=iMap[item] if item in iMap else -1
    rating=alpha
    rating = np.inner(y_u[uMap[i]],y_i[iMap[j]]) + alpha  + (uB[i] if i in uB else 0)  + (iB[j] if j in iB else 0) 
    return rating  

def getPrediction2(alpha,uB,iB,i,j,y_u,y_i,uMap,iMap):
    rating = alpha  + (uB[i] if i in uB else 0)  + (iB[j] if j in iB else 0)
    if i in uMap and j in iMap:
        rating +=np.inner(y_u[uMap[i]],y_i[iMap[j]]) 
    return rating  

def getPrediction3(alpha,uB,iB,i,j,y_u,y_i,y_i2,uMap,iMap):
    rating = alpha  + (uB[i] if i in uB else 0)  + (iB[j] if j in iB else 0)
    if i in uMap and j in iMap:
        rating +=np.inner(y_u[uMap[i]],y_i[iMap[j]]) 
        rating +=np.inner(y_u[uMap[i]],y_i2[iMap[j]]) 
    return rating   

def getWordCleaner():
    tokenizer = RegexpTokenizer(r'\w+')
    en_stop = get_stop_words('en')
    en_stop=set(en_stop)
    en_stop.update(['.', ',', '"', "'", '?', '!', ':', ';', '(', ')', '[', ']', '{', '}'])
    p_stemmer = PorterStemmer()
    return tokenizer,en_stop,p_stemmer

def trainLDA(rData,latentFactor=50,numOfTokens=15000,iterations=40,savePath="dataset/lda2_"):
    # latentFactor=50
    tokenizer,en_stop,p_stemmer=getWordCleaner()
    texts=[]
    text=rData['text'].values
    nounList=[]
    print("Start lemmatizing")
    for d in text:    
        d = d.lower()
        tokens = tokenizer.tokenize(d)
        stopped_tokens = [i for i in tokens if not i in en_stop]
#         tagged_text = nltk.pos_tag(stopped_tokens)
#         for word, tag in tagged_text:
#             if tag in ["NN", "NNS"]:
#                 nounList.append(word)
#         stopped_tokens=nounList
#     stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
        lemm_tokens = [lemm.lemmatize(i) for i in stopped_tokens if(len(i)>2)]
        texts.append(lemm_tokens)
    print("finished lemmatizing")
    dictionary = corpora.Dictionary(texts)
    dictionary.filter_extremes(keep_n=numOfTokens)
    corpora.Dictionary.save(dictionary, savePath+str(latentFactor)+"_factor.dict")
    print("Dictionary Created and Saved to: "+savePath)
    corpus = [dictionary.doc2bow(t) for t in texts]
    ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=latentFactor, id2word = dictionary, passes=iterations)
    ldamodel.save(savePath+str(latentFactor)+"_factor.lda")
    print("LDA Model Saved")
    return ldamodel,dictionary

def initMaps(tData,vData):
    uTrainDict = defaultdict(lambda: defaultdict(int))
    iTrainDict = defaultdict(lambda: defaultdict(int))
    uValidDict = defaultdict(lambda: defaultdict(int))
    iValidDict = defaultdict(lambda: defaultdict(int))
    uMap = defaultdict(int)
    uCount=0
    iMap = defaultdict(int)
    iCount=0
    for i in tData:
        user, item, rating = i['user_id'], i['business_id'], i['stars']
        uTrainDict[user][item] = rating
        iTrainDict[item][user] = rating
        if user not in uMap:
            uMap[user]=uCount
            uCount+=1
        if item not in iMap:
            iMap[item]=iCount
            iCount+=1
    for i in vData:
        user, item, rating = i['user_id'], i['business_id'], i['stars']
        uValidDict[user][item] = rating
    return uTrainDict,iTrainDict,uMap,iMap,uValidDict

In [48]:
def learnItemTopics(rData,ldamodel,dictionary,iMap,latentFactor=50):
    tokenizer,en_stop,p_stemmer=getWordCleaner()
    y_i=np.zeros((len(iMap),latentFactor ))
    # print(y_i)
    testbCount=0
    for b in iMap:
        #     print(b)
        nounList=[]
        for d in rData[rData.business_id==b]['text'].values:
            d=d.lower()
            tokens = tokenizer.tokenize(d)
            stopped_tokens = [i for i in tokens if not i in en_stop]
#             tagged_text = nltk.pos_tag(stopped_tokens)
#             for word, tag in tagged_text:
#                 if tag in ["NN", "NNS"]:
#                     nounList.append(word)
#             stopped_tokens=nounList
            lemm_tokens = [lemm.lemmatize(i) for i in stopped_tokens if(len(i)>2)]
            bow = dictionary.doc2bow(lemm_tokens)
            topics=ldamodel.get_document_topics(bow)
            for topic,prob in topics:
                y_i[iMap[b]][topic]=prob
        testbCount+=1
        if(testbCount%1000==0):
            print(testbCount)
    return y_i

def getTopics(rData,ldamodel,dictionary,iMap,latentFactor=50):
    tokenizer,en_stop,p_stemmer=getWordCleaner()
    y_i=np.zeros((len(iMap),latentFactor ))
    # print(y_i)
    testbCount=0
    text=""
    for b in iMap:
        for d in rData[rData.business_id==b]['text'].values:
            d=d.lower()
            text+=d
        testbCount+=1
        if(testbCount%1000==0):
            print(testbCount)
    tokens = tokenizer.tokenize(text)
    stopped_tokens = [i for i in tokens if not i in en_stop]
    lemm_tokens = [lemm.lemmatize(i) for i in stopped_tokens if(len(i)>2)]
    bow = dictionary.doc2bow(lemm_tokens)
    topics=ldamodel.get_document_topics(bow)
    return topics

def train_LDA_LFM(lam,tData,vData,factor,trials,rData,ldamodel,dictionary):
    print("initializing")
    uTrainDict,iTrainDict,uMap,iMap,uValidDict=initMaps(tData,vData)
    print("initializing done ",len(uTrainDict),len(iTrainDict),len(uMap),len(iMap),len(uValidDict))
    uB = defaultdict(float)
    iB = defaultdict(float)
    latentFactor=factor
    y_u=np.random.normal(scale=1./latentFactor,size=(len(uTrainDict),latentFactor ))
    print("Learning Item Topics "," ",factor," ",trials)
    y_i=learnItemTopics(rData,ldamodel,dictionary,iMap,latentFactor=factor)
    print("Topic Learnt ",len(y_i)," ",factor," ",trials)
    alpha = 0
    totalTrials=trials
    for counter in range(totalTrials):
        alpha=0
        for i in uTrainDict:
            for j in uTrainDict[i]:
                alpha += uTrainDict[i][j] - uB[i] -iB[j] - np.inner(y_u[uMap[i]],y_i[iMap[j]])
        alpha /= len(tData)
        print(alpha)
        for i in uTrainDict:
            uB[i] = 0
            for j in uTrainDict[i]:
                uB[i] += uTrainDict[i][j]  - alpha - iB[j] - np.inner(y_u[uMap[i]],y_i[iMap[j]])
            uB[i] /= (lam + len(uTrainDict[i])) 
        for j in iTrainDict:
            iB[j] = 0
            for i in iTrainDict[j]:
                iB[j] += iTrainDict[j][i]  -alpha - uB[i] - np.inner(y_u[uMap[i]],y_i[iMap[j]])
            iB[j] /= (lam + len(iTrainDict[j]))
    
        for i in uTrainDict:
            for lf in range(latentFactor):
                y_u[uMap[i]][lf] = 0
                for j in uTrainDict[i]:
                    y_u[uMap[i]][lf] += y_i[iMap[j]][lf]*(uTrainDict[i][j]  - alpha - iB[j]  +y_i[iMap[j]][lf]*y_i[iMap[j]][lf]-np.inner(y_u[uMap[i]],y_i[iMap[j]]) )
                    y_u[uMap[i]][lf]  /= (lam + y_i[iMap[j]][lf]*y_i[iMap[j]][lf])
    vMSE = 0
    for i in uValidDict:
        for j in uValidDict[i]:
#             vMSE += ((alpha  + (uB[i] if i in uB else 0)  + (iB[j] if j in iB else 0) - uValidDict[i][j]) **2)
            vMSE += ((getPrediction2(alpha,uB,iB,i,j,y_u,y_i,uMap,iMap) - uValidDict[i][j]) **2)
    vMSE /= len(vData)
    print (vMSE)
    return vMSE,alpha,uB,iB,uMap,iMap

def train_LDA_LFM2(lam,tData,vData,factor,trials,rData,ldamodel,dictionary):
    print("initializing")
    uTrainDict,iTrainDict,uMap,iMap,uValidDict=initMaps(tData,vData)
    print("initializing done ",len(uTrainDict),len(iTrainDict),len(uMap),len(iMap),len(uValidDict))
    uB = defaultdict(float)
    iB = defaultdict(float)
    latentFactor=factor
    y_u=np.random.normal(scale=1./latentFactor,size=(len(uTrainDict),latentFactor ))
    print("Learning Item Topics "," ",factor," ",trials)
    y_i=learnItemTopics(rData,ldamodel,dictionary,iMap,latentFactor=factor)
    y_i2=np.random.normal(scale=1./latentFactor,size=(len(iTrainDict),latentFactor ))
    print("Topic Learnt ",len(y_i)," ",factor," ",trials)
    alpha = 0
    totalTrials=trials
    for counter in range(totalTrials):
        alpha=0
        for i in uTrainDict:
            for j in uTrainDict[i]:
                alpha += uTrainDict[i][j] - uB[i] -iB[j] - np.inner(y_u[uMap[i]],y_i[iMap[j]]) - np.inner(y_u[uMap[i]],y_i2[iMap[j]])
        alpha /= len(tData)
        print(alpha)
        for i in uTrainDict:
            uB[i] = 0
            for j in uTrainDict[i]:
                uB[i] += uTrainDict[i][j]  - alpha - iB[j] - np.inner(y_u[uMap[i]],y_i[iMap[j]]) - np.inner(y_u[uMap[i]],y_i2[iMap[j]])
            uB[i] /= (lam + len(uTrainDict[i])) 
        for j in iTrainDict:
            iB[j] = 0
            for i in iTrainDict[j]:
                iB[j] += iTrainDict[j][i]  -alpha - uB[i] - np.inner(y_u[uMap[i]],y_i[iMap[j]]) - np.inner(y_u[uMap[i]],y_i2[iMap[j]])
            iB[j] /= (lam + len(iTrainDict[j]))
    
        for i in uTrainDict:
            for lf in range(latentFactor):
                y_u[uMap[i]][lf] = 0
                for j in uTrainDict[i]:
                    y_u[uMap[i]][lf] += (y_i[iMap[j]][lf]+y_i2[iMap[j]][lf])*(uTrainDict[i][j]  - alpha - iB[j]  +y_i[iMap[j]][lf]*y_i[iMap[j]][lf]-np.inner(y_u[uMap[i]],y_i[iMap[j]]) +y_i2[iMap[j]][lf]*y_i2[iMap[j]][lf]-np.inner(y_u[uMap[i]],y_i2[iMap[j]]))
                    y_u[uMap[i]][lf]  /= (lam + y_i[iMap[j]][lf]*y_i[iMap[j]][lf] + y_i2[iMap[j]][lf]*y_i2[iMap[j]][lf])
    
        for j in iTrainDict:
            for lf in range(latentFactor):
                y_i2[iMap[j]][lf] = 0
                for i in iTrainDict[j]:
                    y_i2[iMap[j]][lf] += y_u[uMap[i]][lf]*(uTrainDict[i][j]  - alpha - uB[i] - np.inner(y_u[uMap[i]],y_i[iMap[j]]) +y_u[uMap[i]][lf]*y_u[uMap[i]][lf] - np.inner(y_u[uMap[i]],y_i2[iMap[j]]) +y_u[uMap[i]][lf]*y_u[uMap[i]][lf] )
                    y_i2[iMap[j]][lf] /= (lam + y_u[uMap[i]][lf]*y_u[uMap[i]][lf])
    vMSE = 0
    for i in uValidDict:
        for j in uValidDict[i]:
#             vMSE += ((alpha  + (uB[i] if i in uB else 0)  + (iB[j] if j in iB else 0) - uValidDict[i][j]) **2)
            vMSE += ((getPrediction3(alpha,uB,iB,i,j,y_u,y_i,y_i2,uMap,iMap) - uValidDict[i][j]) **2)
    vMSE /= len(vData)
    print (vMSE)
    return vMSE,alpha,uB,iB,uMap,iMap

def findLam_Fact(lam,tData,vData,factor,trials):
    uTrainDict,iTrainDict,uMap,iMap,uValidDict=initMaps(tData,vData)
    uB = defaultdict(float)
    iB = defaultdict(float)
    latentFactor=factor
    y_u=np.random.normal(scale=1./latentFactor,size=(len(uTrainDict),latentFactor ))
    y_i=np.random.normal(scale=1./latentFactor,size=(len(iTrainDict),latentFactor ))
    alpha = 0
    totalTrials=trials
    for counter in range(totalTrials):
        alpha=0
        for i in uTrainDict:
            for j in uTrainDict[i]:
                alpha += uTrainDict[i][j] - uB[i] -iB[j] - np.inner(y_u[uMap[i]],y_i[iMap[j]])
        alpha /= len(tData)
        print(alpha)
        for i in uTrainDict:
            uB[i] = 0
            for j in uTrainDict[i]:
                uB[i] += uTrainDict[i][j]  - alpha - iB[j] - np.inner(y_u[uMap[i]],y_i[iMap[j]])
            uB[i] /= (lam + len(uTrainDict[i])) 
        for j in iTrainDict:
            iB[j] = 0
            for i in iTrainDict[j]:
                iB[j] += iTrainDict[j][i]  -alpha - uB[i] - np.inner(y_u[uMap[i]],y_i[iMap[j]])
            iB[j] /= (lam + len(iTrainDict[j]))
    
        for i in uTrainDict:
            for lf in range(latentFactor):
                y_u[uMap[i]][lf] = 0
                for j in uTrainDict[i]:
                    y_u[uMap[i]][lf] += y_i[iMap[j]][lf]*(uTrainDict[i][j]  - alpha - iB[j]  +y_i[iMap[j]][lf]*y_i[iMap[j]][lf]-np.inner(y_u[uMap[i]],y_i[iMap[j]]) )
                    y_u[uMap[i]][lf]  /= (lam + y_i[iMap[j]][lf]*y_i[iMap[j]][lf])
    #     print(y_u)
        for j in iTrainDict:
            for lf in range(latentFactor):
                y_i[iMap[j]][lf] = 0
                for i in iTrainDict[j]:
                    y_i[iMap[j]][lf] += y_u[uMap[i]][lf]*(uTrainDict[i][j]  - alpha - uB[i] - np.inner(y_u[uMap[i]],y_i[iMap[j]]) +y_u[uMap[i]][lf]*y_u[uMap[i]][lf] )
                    y_i[iMap[j]][lf] /= (lam + y_u[uMap[i]][lf]*y_u[uMap[i]][lf])
    vMSE = 0
    for i in uValidDict:
        for j in uValidDict[i]:
#             vMSE += ((alpha  + (uB[i] if i in uB else 0)  + (iB[j] if j in iB else 0) - uValidDict[i][j]) **2)
            vMSE += ((getPrediction2(alpha,uB,iB,i,j,y_u,y_i,uMap,iMap) - uValidDict[i][j]) **2)
    vMSE /= len(vData)
    print (vMSE)
    return vMSE,alpha,uB,iB,uMap,iMap
print("done")

done


In [58]:
bdata=rd.readData(fileName=businessFile,breakCondition=5000000)



Data Loaded
Total Data:  156639


In [70]:
categoryDict=defaultdict(int)
businessSet=set()
for b in bdata:
    bid,categoryList,state=b['business_id'],b['categories'],b['state']
    if 'Restaurants' in categoryList and state=='ON':
        businessSet.add(bid)
print(len(businessSet))
    

12634


In [74]:
categoryDict=defaultdict(int)
businessSet=set()
for b in bdata:
    bid,categoryList,state=b['business_id'],b['categories'],b['state']
    if 'Restaurants' in categoryList and state=='AZ':
        businessSet.add(bid)
print(len(businessSet))

10219


In [75]:
rawdata=rd.readData(fileName=reviewFile,breakCondition=5000000)
data=[]
for d in rawdata:
    bid=d['business_id']
    if bid in businessSet:
        data.append(d)
print(len(data))
rawdata=[]

Data Loaded
Total Data:  4736897
837240


In [76]:
dictionary_path="dataset/lda2_400000_50_factor.dict"
dictionary = corpora.Dictionary.load(dictionary_path)

In [77]:
ldapath="dataset/lda2_400000_50_factor.lda"
ldaModel = gensim.models.ldamodel.LdaModel.load(ldapath)



In [67]:
latentfactor=50
trials=30
trainSize=100000
tData=data[:trainSize]
vData=data[trainSize:trainSize+100000]
fileSavePath="dataset/lda2_"+str((trainSize))+"_"
rData=pd.DataFrame(tData)
uTrainDict,iTrainDict,uMap,iMap,uValidDict=initMaps(tData,vData)

In [54]:
tData=data[:trainSize]
print("Training Started: ",len(tData))
lamdas=[4.5]
trials=[2]
factors=[latentfactor]
# factors=[1,2,4,6,8,10,15,20,25,30,35,40,45,50]
vMSE = np.iinfo(np.int32).max
bestLam=0.01
bestTrial=1
bestFactor=1
vMSEList=[]
trialList=[]
factorList=[]
lamdaList=[]
for lam in lamdas:
    tempvMSE=1
    for t in trials:
        for f in factors:
            tempvMSE,alpha,uB,iB,uMap,iMap=train_LDA_LFM2(lam,tData,vData,f,t,rData,ldaModel,dictionary)
#             tempvMSE,alpha,uB,iB,uMap,iMap=findLam_Fact(lam,tData,vData,f,t)
#             vMSEList.append(tempvMSE)
            print ("----------lamda: "+str(lam)+"-----------Trails: "+str(t)+"-------------Factor: "+str(f)+" MSE: "+str(tempvMSE))
            if(tempvMSE<vMSE):
                vMSE=tempvMSE
                bestLam=lam
                bestTrial=t
                bestFactor=f
                bestAlpha=alpha
                bestuB=uB
                bestiB=iB
            vMSEList.append(tempvMSE)
            factorList.append(f)
#     vMSEList.append(tempvMSE)
#     lamdaList.append(i)
        
# import matplotlib.pyplot as plt
# plt.scatter(factorList,vMSEList,color='red',marker='^')
# plt.xlabel('Factors')
# plt.ylabel('MSE')
# plt.show()

print("Best Value for Lamda is: ",bestLam," vMSE: ",vMSE)            

Training Started:  400000
initializing
initializing done  152212 10095 152212 10095 97945
Learning Item Topics    50   2
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
Topic Learnt  10095   50   2
3.73856741363
3.71115990778
1.58022116302
----------lamda: 4.5-----------Trails: 2-------------Factor: 50 MSE: 1.58022116302
Best Value for Lamda is:  4.5  vMSE:  1.58022116302


In [55]:
tData=data[:trainSize]
print("Training Started: ",len(tData))
lamdas=[4.5]
trials=[2]
factors=[latentfactor]
# factors=[1,2,4,6,8,10,15,20,25,30,35,40,45,50]
vMSE = np.iinfo(np.int32).max
bestLam=0.01
bestTrial=1
bestFactor=1
vMSEList=[]
trialList=[]
factorList=[]
lamdaList=[]
for lam in lamdas:
    tempvMSE=1
    for t in trials:
        for f in factors:
            tempvMSE,alpha,uB,iB,uMap,iMap=train_LDA_LFM(lam,tData,vData,f,t,rData,ldaModel,dictionary)
#             tempvMSE,alpha,uB,iB,uMap,iMap=findLam_Fact(lam,tData,vData,f,t)
#             vMSEList.append(tempvMSE)
            print ("----------lamda: "+str(lam)+"-----------Trails: "+str(t)+"-------------Factor: "+str(f)+" MSE: "+str(tempvMSE))
            if(tempvMSE<vMSE):
                vMSE=tempvMSE
                bestLam=lam
                bestTrial=t
                bestFactor=f
                bestAlpha=alpha
                bestuB=uB
                bestiB=iB
            vMSEList.append(tempvMSE)
            factorList.append(f)
#     vMSEList.append(tempvMSE)
#     lamdaList.append(i)
        
# import matplotlib.pyplot as plt
# plt.scatter(factorList,vMSEList,color='red',marker='^')
# plt.xlabel('Factors')
# plt.ylabel('MSE')
# plt.show()

print("Best Value for Lamda is: ",bestLam," vMSE: ",vMSE)            

Training Started:  400000
initializing
initializing done  152212 10095 152212 10095 97945
Learning Item Topics    50   2
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
Topic Learnt  10095   50   2
3.7385810241
3.7115155972
1.57939785252
----------lamda: 4.5-----------Trails: 2-------------Factor: 50 MSE: 1.57939785252
Best Value for Lamda is:  4.5  vMSE:  1.57939785252


In [105]:
print("RMSE: ",math.sqrt(vMSE)) 

RMSE:  1.255316944411388


In [56]:
tData=data[:trainSize]
print("Training Started: ",len(tData))
lamdas=[4.5]
trials=[2]
factors=[latentfactor]
# factors=[1,2,4,6,8,10,15,20,25,30,35,40,45,50]
vMSE = np.iinfo(np.int32).max
bestLam=0.01
bestTrial=1
bestFactor=1
vMSEList=[]
trialList=[]
factorList=[]
lamdaList=[]
for lam in lamdas:
    tempvMSE=1
    for t in trials:
        for f in factors:
            tempvMSE,alpha,uB,iB,uMap,iMap=findLam_Fact(lam,tData,vData,f,t)
#             tempvMSE,alpha,uB,iB,uMap,iMap=findLam_Fact(lam,tData,vData,f,t)
#             vMSEList.append(tempvMSE)
            print ("----------lamda: "+str(lam)+"-----------Trails: "+str(t)+"-------------Factor: "+str(f)+" MSE: "+str(tempvMSE))
            if(tempvMSE<vMSE):
                vMSE=tempvMSE
                bestLam=lam
                bestTrial=t
                bestFactor=f
                bestAlpha=alpha
                bestuB=uB
                bestiB=iB
            vMSEList.append(tempvMSE)
            factorList.append(f)
#     vMSEList.append(tempvMSE)
#     lamdaList.append(i)
        
# import matplotlib.pyplot as plt
# plt.scatter(factorList,vMSEList,color='red',marker='^')
# plt.xlabel('Factors')
# plt.ylabel('MSE')
# plt.show()

print("Best Value for Lamda is: ",bestLam," vMSE: ",vMSE)            

Training Started:  400000
3.73858965774
3.72182301938
1.57582063093
----------lamda: 4.5-----------Trails: 2-------------Factor: 50 MSE: 1.57582063093
Best Value for Lamda is:  4.5  vMSE:  1.57582063093
